In [1]:
from PIL import Image, ImageDraw 
from IPython.display import Image as DImage
import sys, os, hashlib, functools

In [2]:
#INTERFACE
pix_mode    = 1 #compare per-pixel
hash_mode   = 1 #compare hashes
string_mode = 1 #compare hashes of rows and columns
black_mode  = 0 #compare only black pixels
bin_ratio   = 200 #binarization threshold (0-255*3)
basewidth   = 8 #image width after compression (px)
resize_mode = 1 #none = 0, antialias = 1, bicubic = 2
hashing_mode = 'md5'#md5 sha1 sha224 sha256 sha384 sha512
rotate_mode = 1 

#init directories
if not os.path.exists(os.getcwd()+'/examples/'):
    os.mkdir('examples')
if not os.path.exists(os.getcwd()+'/binary/'):
    os.mkdir('binary')
if not os.path.exists(os.getcwd()+'/resized/'):
    os.mkdir('resized')
examples_path = os.getcwd()+'/examples/'
binary_path = os.getcwd()+'/binary/'
resized_path = os.getcwd()+'/resized/'

#global variables
names = []
values = []
distances = []
strings = []
results = []

RESULT = ''

In [3]:
input_address = input('Input image address:') #input must be in "" ("test.jpg")

Input image address:"test.jpg"


In [4]:
def address_helper(input_img_addr, mode):
    if mode == 'png':
        input_img_addr = input_img_addr.replace('.jpg','.png')
        input_img_addr = input_img_addr.replace('.gif','.png')
    if mode == 'bin':
        input_img_addr = input_img_addr.replace('/examples/','/binary/')
        input_img_addr = input_img_addr.replace('/resized/','/binary/')
    if mode == 'res':
        input_img_addr = input_img_addr.replace('/examples/','/resized/')
        input_img_addr = input_img_addr.replace('/binary/','/resized/')
    if mode == 'cln':
        input_img_addr = input_img_addr.replace('.jpg','')
        input_img_addr = input_img_addr.replace('.gif','')
        input_img_addr = input_img_addr.replace('.png','')
        input_img_addr = input_img_addr.replace(os.getcwd()+'/binary/','')
        input_img_addr = input_img_addr.replace(os.getcwd()+'/resized/','')
    return input_img_addr

In [5]:
def binarization(input_img_addr):
    input_img = Image.open(input_img_addr) 
    draw = ImageDraw.Draw(input_img)
    input_width = input_img.size[0]
    input_height = input_img.size[1]
    input_pixels = input_img.load()
    
    if ".gif" not in input_img_addr:
        for i in range(input_width):
            for j in range(input_height):
                r = input_pixels[i, j][0]
                g = input_pixels[i, j][1]
                b = input_pixels[i, j][2]
                s = r + g + b
                if (s > bin_ratio):
                    r, g, b = 255, 255, 255
                else:
                    r, g, b = 0, 0, 0
                draw.point((i, j), (r, g, b))
    
    input_img_addr = address_helper(input_img_addr,'png')
    input_img.save(address_helper(input_img_addr,'bin'), "PNG")
    del draw

In [6]:
# добавить сохранение еще трех поворотов в папку бинари
def saverotate(input_img_addr):
    input_img_addr = address_helper(input_img_addr,'png')
    
    input_img = Image.open(input_img_addr).rotate(90)
    input_img_addr = input_img_addr.replace('.','90.')
    input_img.save(input_img_addr, "PNG")
   
    input_img = Image.open(input_img_addr).rotate(90)
    input_img_addr = input_img_addr.replace('90.','180.')
    input_img.save(input_img_addr, "PNG")
   
    input_img = Image.open(input_img_addr).rotate(90)
    input_img_addr = input_img_addr.replace('180.','270.')
    input_img.save(input_img_addr, "PNG")

In [7]:
def gethash(input_img_addr):
    input_img = Image.open(input_img_addr).convert('L') #convert 'tuple' to 'float', image is already binarized
    avg = functools.reduce(lambda x, y: x + y, input_img.getdata())/ 64.
    return functools.reduce(lambda x, yz: x | (yz[1] << yz[0]), 
                            enumerate(map(lambda i: 0 if i < avg else 1, input_img.getdata())),0)

In [8]:
def hamdist(input_img_hash, example_hash):
    distance, i = 0, input_img_hash ^ example_hash
    while i:
        distance += 1
        i &= i - 1
    return distance

In [9]:
def shamdist(str1, str2, prevMin=None):
    diffs = 0
    if len(str1) != len(str2):
        return max(len(str1),len(str2))
    for ch1, ch2 in zip(str1, str2):
        if ch1 != ch2:
            diffs += 1
            if prevMin is not None and diffs > prevMin:
                return None
    return diffs

In [10]:
def resize(input_img_addr, basewidth):
    input_img = Image.open(input_img_addr)
    wpercent = (basewidth / float(input_img.size[0]))
    hsize = int((float(input_img.size[1]) * float(wpercent)))
    if resize_mode == 1:
        input_img = input_img.resize((basewidth,hsize), Image.ANTIALIAS)
    if resize_mode == 2:
        input_img = input_img.resize((basewidth,hsize), Image.BICUBIC)
    else:
        input_img = input_img.resize((basewidth,hsize))
    input_img.save(address_helper(input_img_addr,'res'), "PNG")

In [11]:
def pix_compare(input_img_addr, example_addr):
    input_img = Image.open(input_img_addr)
    input_pixels = input_img.load()
    example = Image.open(example_addr)
    true = 0 
    
    for i in range(0, input_img.size[0]):
        for j in range(0, input_img.size[1]):
            input_pixel = input_img.getpixel((i, j))
            example_pixel = example.getpixel((i, j))
            if input_pixel == example_pixel:                        
                if not (black_mode == 1 and input_pixels[i, j][0] == 0):
                    true+=1  
    return true

In [12]:
def string_compare(input_img_addr, example_addr):
    input_img = Image.open(input_img_addr)
    example = Image.open(example_addr)
    true = 0
    input_pix = ''
    example_pix = ''
    
    for i in range(0, input_img.size[0]):
        for j in range(0, input_img.size[1]):
            input_pix = input_pix + str(input_img.getpixel((i, j)))
            example_pix = example_pix + str(example.getpixel((i, j)))
        if hashing_mode == 'md5':
            true+=shamdist(hashlib.md5(input_pix).hexdigest(),hashlib.md5(example_pix).hexdigest())
        if hashing_mode == 'sha1':
            true+=shamdist(hashlib.sha1(input_pix).hexdigest(),hashlib.sha1(example_pix).hexdigest())
        if hashing_mode == 'sha224':
            true+=shamdist(hashlib.sha224(input_pix).hexdigest(),hashlib.sha224(example_pix).hexdigest())
        if hashing_mode == 'sha256':
            true+=shamdist(hashlib.sha256(input_pix).hexdigest(),hashlib.sha256(example_pix).hexdigest())
        if hashing_mode == 'sha384':
            true+=shamdist(hashlib.sha384(input_pix).hexdigest(),hashlib.sha384(example_pix).hexdigest())
        if hashing_mode == 'sha512':
            true+=shamdist(hashlib.sha512(input_pix).hexdigest(),hashlib.sha512(example_pix).hexdigest())
        input_pix = ''
        example_pix = ''
        
    for j in range(0, input_img.size[1]):
        for i in range(0, input_img.size[0]):
            input_pix = input_pix + str(input_img.getpixel((i, j)))
            example_pix = example_pix + str(example.getpixel((i, j)))          
        true+=shamdist(hashlib.md5(input_pix).hexdigest(),hashlib.md5(example_pix).hexdigest())
        input_pix = ''
        example_pix = ''
        
    return true

In [13]:
def compare1to1(input_img_addr, example_addr):
    example_addr = address_helper(example_addr,'png')
    input_img_addr = os.getcwd()+'/binary/'+address_helper(input_img_addr,'png')
    
    if pix_mode == 1:      
        values.append(pix_compare(input_img_addr, example_addr))
        
    if string_mode == 1:
        strings.append(string_compare(input_img_addr, example_addr))
        
    if hash_mode == 1:
        resize(input_img_addr, basewidth)
        resize(example_addr, basewidth)
        
        distances.append(hamdist(gethash(address_helper(example_addr,'res')), 
                                 gethash(address_helper(input_img_addr,'res'))))
            
    example_addr = address_helper(example_addr, 'cln')
    names.append(example_addr)   

In [14]:
def compare1toN(input_img_addr):
    #delete previous values
    global names
    global values
    global distances
    global strings
    names = []
    values = []
    distances = []
    strings = []
    
    examples = os.listdir(examples_path)
    for i in range(len(examples)):
        if ".gif" in examples[i] or ".jpg" in examples[i] or ".png" in examples[i]:
            binarization(examples_path+examples[i])
            if rotate_mode == 1:
                saverotate(binary_path+examples[i])

    examples = os.listdir(binary_path)
    for i in range(len(examples)):
        if ".gif" in examples[i] or ".jpg" in examples[i] or ".png" in examples[i]:
            compare1to1(input_img_addr,binary_path+examples[i])

In [15]:
def print1toN():
    if pix_mode == 1:
        print('Pixel mode:')
        for i in range(len(names)):
            print(names[i],values[i])
    if hash_mode == 1:
        print('Hash mode:')
        for i in range(len(names)):
            print(names[i],distances[i])
    if string_mode == 1:
        print('String mode:')
        for i in range(len(names)):
            print(names[i],strings[i])

In [16]:
def result1toN():
    print('Results:')
    global results
    global RESULT
    results = ['']
    if pix_mode == 1 and values != []:
        results.append(names[values.index(max(values))])
        print('Pixel mode:',names[values.index(max(values))])
    if hash_mode == 1 and distances != []:
        results.append(names[distances.index(min(distances))])
        print('Hash mode:',names[distances.index(min(distances))])
    if string_mode == 1 and strings != []:
        results.append(names[strings.index(min(strings))])
        print('String mode:',names[strings.index(min(strings))])
    RESULT += max(set(results), key=results.count)
    print('Result:',RESULT)

In [17]:
binarization(input_address)
compare1toN(input_address)
print1toN()
result1toN()

Pixel mode:
('compare', 44884)
('ctest180', 23908)
('compare90', 31278)
('b270', 30071)
('test180', 29892)
('test90', 30397)
('compare180', 30901)
('a', 47718)
('compare270', 31155)
('b', 47416)
('ctest', 38706)
('b180', 29774)
('b90', 30263)
('ctest90', 23905)
('ctest270', 24434)
('test', 47718)
('a90', 30397)
('a270', 30146)
('test270', 30146)
('a180', 29892)
Hash mode:
('compare', 8)
('ctest180', 34)
('compare90', 46)
('b270', 41)
('test180', 48)
('test90', 42)
('compare180', 50)
('a', 0)
('compare270', 42)
('b', 2)
('ctest', 23)
('b180', 48)
('b90', 42)
('ctest90', 35)
('ctest270', 31)
('test', 0)
('a90', 42)
('a270', 41)
('test270', 41)
('a180', 48)
String mode:
('compare', 13063)
('ctest180', 13171)
('compare90', 13123)
('b270', 13187)
('test180', 13175)
('test90', 13159)
('compare180', 13162)
('a', 0)
('compare270', 13189)
('b', 1645)
('ctest', 9560)
('b180', 13160)
('b90', 13148)
('ctest90', 13135)
('ctest270', 13184)
('test', 0)
('a90', 13159)
('a270', 13182)
('test270', 13182